In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier

import random
import numpy as np
import torch

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import optuna

import random
import numpy as np
from collections import Counter
import torch
import torch.nn.functional as F
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/mnt/storageG1/lwang/miniconda3/envs/new-ml/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


Using device: cuda


/mnt/storageG1/lwang/miniconda3/envs/new-ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import random
import numpy as np

# If using PyTorch
import torch

# If using TensorFlow

# Optional: If using Python hash-based functions
os.environ["PYTHONHASHSEED"] = "42"

# Set seed for base Python random
random.seed(42)

# Set seed for NumPy
np.random.seed(42)

# Set seed for PyTorch (CPU and GPU)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # if using multi-GPU

# Force deterministic behavior in PyTorch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


#### raw data gen

In [3]:
import torch
import pandas as pd
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Load only positive (AMP) sequences
adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")

unique_letters = set(''.join(adam_df["Sequence"]))
print(unique_letters)
print(len(unique_letters))
print(f"Number of sequences after filtering: {len(adam_df)}")
adam_df = adam_df.drop_duplicates(subset='Sequence')
tb_df = pd.read_csv('../data/all_seq702.csv')
adam_df = adam_df[~adam_df['Sequence'].isin(tb_df['Sequences'])]
adam_df = adam_df[adam_df["Sequence"].str.len() >= 10]
generation_seqs = adam_df["Sequence"].reset_index(drop=True)

import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]  # all residues except the last
        target_seq = seq[1:]  # all residues except the first
        length = len(input_seq.replace("X", ""))  # unpadded length
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        # target_indices = torch.tensor(["ACDEFGHIKLMNPQRSTVWY".index(res) for res in target_seq], dtype=torch.long)
        return input_one_hot, target_one_hot, length

def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)

    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]

    sequences = [seq.T for seq in sequences]  # transpose to [seq_len, features]
    targets = [tgt.T for tgt in targets]      # transpose targets as well

    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)

    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)

    return packed_input, packed_target, lengths


# Train/val/test split
train_seqs, test_seqs = train_test_split(generation_seqs, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)

train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Dataset sizes
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:", dataset_sizes)

for x, y, l in train_loader:
    print("Input shape:", x.data.shape)  # [L, B, 20]
    print("Target shape:", y.data.shape)  # [L, B, 20]
    print("Lengths:", y.batch_sizes)  # Lengths of sequences in the batch
    break

{'G', 'F', 'L', 'Q', 'D', 'H', 'A', 'N', 'V', 'W', 'C', 'M', 'S', 'K', 'E', 'P', 'Y', 'I', 'R', 'T'}
20
Number of sequences after filtering: 3306
Dataset sizes: {'Train': 2219, 'Validation': 475, 'Test': 476}
Input shape: torch.Size([1793, 20])
Target shape: torch.Size([1793, 20])
Lengths: tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 59, 57, 55, 53, 53, 50,
        46, 45, 40, 40, 38, 35, 32, 31, 31, 30, 29, 29, 28, 26, 20, 19, 16, 14,
        13, 10,  7,  5,  5,  4,  4,  4,  4,  3,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])


## tb amp

In [4]:
df = pd.read_csv('../data/all_seq702.csv')
df = df.drop_duplicates(subset='Sequences')

max_length = df['Sequences'].str.len().max()
print(max_length)
# df['Sequences'] = df['Sequences'].apply(lambda x: x.ljust(max_length, 'X'))

unique_letters = set(''.join(df["Sequences"]))
print(unique_letters)
print(len(unique_letters))
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
non_standard_amino_acids = unique_letters - amino_acids
print(non_standard_amino_acids)
b_count = df["Sequences"].str.count('B').sum()
print(f"Number of 'B' values: {b_count}")
# manually replaced one of the B with D and the other with N

X = df["Sequences"]
y = df["AMP"]

# df_filtered = df[
#     (df['Sequences'].str.len() >= 10) &
#     (df['Sequences'].apply(lambda x: len(set(x)) > 1)) &
#     (~df['Sequences'].str.contains('X'))
# ]
df_filtered = df

def split_sequence(seq, chunk_size=20):
    return [seq[i:i+chunk_size] for i in range(0, len(seq), chunk_size)]

new_rows = []
for _, row in df_filtered.iterrows():
    seq = row['Sequences']
    amp_label = row['AMP']
    if len(seq) > 40:
        for chunk in split_sequence(seq, 20):
            new_rows.append({'Sequences': chunk, 'AMP': amp_label})
    else:
        new_rows.append({'Sequences': seq, 'AMP': amp_label})

df_filtered = pd.DataFrame(new_rows)


df_filtered = df_filtered[
    (df_filtered['Sequences'].str.len() >= 10) &
    (df_filtered['Sequences'].apply(lambda x: len(set(x)) > 1)) &
    (~df_filtered['Sequences'].str.contains('X'))
]
df_filtered = df_filtered[df_filtered['AMP']==1]
df_filtered = df_filtered.drop_duplicates(subset='Sequences')


128
{'G', 'F', 'L', 'Q', 'D', 'H', 'A', 'N', 'V', 'W', 'C', 'M', 'S', 'K', 'X', 'E', 'P', 'Y', 'I', 'R', 'T'}
21
{'X'}
Number of 'B' values: 0


In [5]:
# Re-import libraries after environment reset
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split

# Load data
df = df_filtered

# Clean and inspect
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
unique_letters = set(''.join(df["Sequences"]))
# non_standard_amino_acids = unique_letters - amino_acids
# df = df[~df["Sequences"].str.contains('|'.join(non_standard_amino_acids))]

# Extract sequences
sequences = df["Sequences"].reset_index(drop=True)

# Define one-hot function
def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

# Define dataset class
class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]
        target_seq = seq[1:]
        length = len(input_seq.replace("X", ""))
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        return input_one_hot, target_one_hot, length

# Define collate function
def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)
    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]
    sequences = [seq.T for seq in sequences]
    targets = [tgt.T for tgt in targets]
    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)
    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)
    return packed_input, packed_target, lengths

# Split and load data
train_seqs, test_seqs = train_test_split(sequences, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)
train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Preview batch
batch_sample = next(iter(train_loader))
batch_sample_shapes = {
    "Input shape": batch_sample[0].data.shape,
    "Target shape": batch_sample[1].data.shape,
    "Lengths": batch_sample[0].batch_sizes
}
batch_sample_shapes


{'Input shape': torch.Size([1179, 20]),
 'Target shape': torch.Size([1179, 20]),
 'Lengths': tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 56, 54, 48, 44, 44, 42, 40, 38, 37,
         35, 18, 17, 17, 16, 15, 11,  9,  9,  9,  9,  6,  6,  6,  6,  5,  2,  2,
          1,  1])}

In [6]:
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:")
for name, size in dataset_sizes.items():
    print(f"{name}: {size}")

Dataset sizes:
Train: 171
Validation: 37
Test: 37


# TB no transfer

In [10]:
# Re-import necessary packages after reset
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
from torch.utils.tensorboard import SummaryWriter

from torch.nn.utils.rnn import pad_packed_sequence


class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GenerativeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Handle packed input
        if isinstance(x, torch.nn.utils.rnn.PackedSequence):
            packed_output, _ = self.lstm(x)
            unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
            return self.fc(unpacked_output)
        else:
            out, _ = self.lstm(x)
            return self.fc(out)

# Criterion

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-notrans-tb/AMP_LSTM_GEN_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])      # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            
            # print('target_shape before reshape',target_seq.shape)
            # target_seq = target_seq.reshape(-1)
            # print(f"Output shape: {output.shape}, Target shape: {target_seq.shape}")

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if train:
                torch.save(model.state_dict(), 'best_model_lstm_generator-notrans-tb.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])  # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            # # target_seq = target_seq.view(-1)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # # target_seq = target_seq.reshape(-1)
            # # target_seq = target_seq.reshape(-1, target_seq.shape[-1])
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            # assert output.size(0) == target_seq.size(0), f"Mismatch: {output.size(0)} vs {target_seq.size(0)}"

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            # print('loss done')
            total_loss += loss.item()
            
            
            preds = output[:, -1, :]        # shape: [B, vocab_size]
            preds = torch.argmax(preds, dim=1)  # shape: [B]

            targets = target_seq[:, -1, :]      # shape: [B, vocab_size]
            targets = torch.argmax(targets, dim=-1)  # shape: [B]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"

    return avg_loss, acc, auc

# Objective for Optuna tuning
def objective_generation(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)

lstm_gen_notrans_tb_best_params = study.best_trial.params
print(lstm_gen_notrans_tb_best_params)

[I 2025-05-02 10:55:14,561] A new study created in memory with name: no-name-f7a5cdc2-a106-4dcf-a41d-0c75ebdcb8f2
[I 2025-05-02 10:55:22,868] Trial 0 finished with value: 2.7173891067504883 and parameters: {'hidden_dim': 144, 'num_layers': 3, 'dropout': 0.15437274023341982, 'lr': 0.005771881998865653, 'weight_decay': 0.0002682713506887346}. Best is trial 0 with value: 2.7173891067504883.
[I 2025-05-02 10:55:25,214] Trial 1 finished with value: 2.7665016651153564 and parameters: {'hidden_dim': 227, 'num_layers': 2, 'dropout': 0.4424763441823395, 'lr': 0.005704011585947213, 'weight_decay': 0.00041701060111245934}. Best is trial 0 with value: 2.7173891067504883.
[I 2025-05-02 10:55:27,423] Trial 2 finished with value: 2.9343535900115967 and parameters: {'hidden_dim': 250, 'num_layers': 1, 'dropout': 0.32673827998102084, 'lr': 0.00154077252086226, 'weight_decay': 0.0008423535539033275}. Best is trial 0 with value: 2.7173891067504883.
[I 2025-05-02 10:55:29,530] Trial 3 finished with value:

{'hidden_dim': 149, 'num_layers': 2, 'dropout': 0.4350981856205326, 'lr': 0.009996913527093828, 'weight_decay': 0.00012991309919791892}


### test

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss
lstm_gen_notrans_tb_best_params = {'hidden_dim': 149, 'num_layers': 2, 'dropout': 0.4350981856205326, 'lr': 0.009996913527093828, 'weight_decay': 0.00012991309919791892}



PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_notrans_tb_best_params["lr"], weight_decay=lstm_gen_notrans_tb_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-notrans-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
    num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
    dropout=lstm_gen_notrans_tb_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")


Epoch 1: Train Loss = 3.0548 | Val Loss = 3.0090 | Acc = 0.0270 | AUC = undefined | Perplexity = 20.2662
Epoch 2: Train Loss = 2.9965 | Val Loss = 2.9435 | Acc = 0.0270 | AUC = undefined | Perplexity = 18.9822
Epoch 3: Train Loss = 2.9277 | Val Loss = 2.8642 | Acc = 0.9730 | AUC = undefined | Perplexity = 17.5345
Epoch 4: Train Loss = 2.8628 | Val Loss = 2.7988 | Acc = 1.0000 | AUC = undefined | Perplexity = 16.4257
Epoch 5: Train Loss = 2.8049 | Val Loss = 2.7393 | Acc = 1.0000 | AUC = undefined | Perplexity = 15.4760
Epoch 6: Train Loss = 2.7479 | Val Loss = 2.6935 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.7829
Epoch 7: Train Loss = 2.6925 | Val Loss = 2.6404 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.0191
Epoch 8: Train Loss = 2.6369 | Val Loss = 2.5883 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.3075
Epoch 9: Train Loss = 2.5930 | Val Loss = 2.5379 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.6535
Epoch 10: Train Loss = 2.5679 | Val Loss = 2.4886 | Acc

### batch gen

In [18]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

aa_vocab = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(aa_vocab)}
idx_to_aa = {i: aa for aa, i in aa_to_idx.items()}

def sample_start_amino_acid():
    return random.choice(amino_acids)

def one_hot_encode_amino_acid(aa, vocab=aa_vocab):
    vec = torch.zeros(len(vocab))
    vec[aa_to_idx[aa]] = 1.0
    return vec


def generate_sequence_from_seed(model, seed, max_length=30, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = [one_hot_encode_amino_acid(aa).to(device) for aa in seed]
    input_tensor = torch.stack(input_seq).unsqueeze(0)  # [1, L, 20]

    generated = seed.copy()

    with torch.no_grad():
        for _ in range(max_length - len(seed)):
            output = model(input_tensor)  # [1, L, vocab]
            logits = output[0, -1, :]  # Last time step → [vocab]

            # Apply temperature and sample
            probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
            next_idx = np.random.choice(len(aa_vocab), p=probs)
            next_aa = idx_to_aa[next_idx]

            # Update sequence
            next_aa_vec = one_hot_encode_amino_acid(next_aa).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 20]
            input_tensor = torch.cat([input_tensor, next_aa_vec], dim=1)
            generated.append(next_aa)

    return ''.join(generated)


class LengthSampler:
    def __init__(self, sequence_lengths):
        """
        Initialize sampler from observed sequence lengths.
        
        Args:
            sequence_lengths (list[int]): List of sequence lengths (e.g., [20, 21, 20, 23, ...])
        """
        self.length_counts = Counter(sequence_lengths)
        self.lengths = np.array(sorted(self.length_counts.keys()))
        counts = np.array([self.length_counts[l] for l in self.lengths])
        self.probs = counts / counts.sum()  # Empirical probabilities

    def sample(self, n=1):
        """
        Sample one or more lengths based on the learned distribution.

        Returns:
            np.ndarray of sampled lengths
        """
        return np.random.choice(self.lengths, size=n, p=self.probs)
length_sampler = LengthSampler([len(seq) for seq in df.loc[df['AMP'] == 1, :]['Sequences']])

# Reload the trained model
gen_model = trained_model
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-notrans.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EYPPGGGYPPGYYGGG
Generated AMP sequence: AHIHPGHKKSGLPG
Generated AMP sequence: KAAYYYHGPKGKGYHPHPPG
Generated AMP sequence: IQYYPYGGGHKHYIHPGYGYRGDYPPRGKGYPHGHHPPY
Generated AMP sequence: IAPRPRKLKGGGPHLYHKR
Generated AMP sequence: FAGGYYPYYGYP
Generated AMP sequence: EVHGGKYYDLKGGHYHPPKYYHKHHG
Generated AMP sequence: VHAHYHGKYHPRHPKPPPGY
Generated AMP sequence: DAAPPGKGPGPKYG
Generated AMP sequence: WAYYHPYPNHGYYP
Generated AMP sequence: QEPYYRPGGYPFGP
Generated AMP sequence: CAKPPGPHHR
Generated AMP sequence: ASPLGIHHYKYPYKPKGPHG
Generated AMP sequence: DAYYYKGYKYYYHKYHGPYP
Generated AMP sequence: HANYGLYQPPHHYYYYLKYP
Generated AMP sequence: IRYHKGPGKPHPGGYKGPRG
Generated AMP sequence: TAAKLPPYLHYHKGGYYPKG
Generated AMP sequence: YAKPPHYPPPKH
Generated AMP sequence: APAGGGYPKGKKGKK
Generated AMP sequence: VLAHPKYPGGPGPYPKPKPY
Generated AMP sequence: HYYGGGGGPGHYGYHG
Generated AMP sequence: VLVPSHGRYYKKRHYYKRRG
Generated AMP sequence: QHSHYKFYGHYYPPKHHPRYWGKGP